keras tuner is a seperate library that helps choose optimal hyperparameters of a TF program: this is called *hyperparameter tuning* or *hypertuning*. there are two different types of hyperparameters
1. **model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
%pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [4]:
import keras_tuner as kt

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0   

using fashion MNIST dataset

when you set up a model for hypertuning you have to define the search space which can be done by: 
- using a model builder function
- or by subclassing the `HyperModel` class of the Keras Tuner API
there are two pre-defined hypermodel classes that can be used in computer vision

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

    return model

the tuner must be instantiated to perform tuning. the keras tuner has four different types. `RandomSearch`, `HyperBand`, `BayesianOptimization`, and `Sklearn`. this will be using the `HyperBand`. 

the hypermodel must be specified. the `objective` (what should be optimized) and the max number of epochs to be trained

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2022-06-16 21:53:49.088128: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + logfactor(max_epochs) and rounding it up to the nearest integer.

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

same args as `tf.keras.model.fit`

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 02m 22s]
val_accuracy: 0.8814166784286499

Best val_accuracy So Far: 0.8868333101272583
Total elapsed time: 00h 16m 54s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 96 and the optimal learning rate for the optimizer
is 0.001.



alright so all that was just finding the best hyperparameters to use for training. now we're solving for the best number of epochs to train with

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5216 - accuracy: 0.8166 - val_loss: 0.4591 - val_accuracy: 0.8315
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3921 - accuracy: 0.8588 - val_loss: 0.4077 - val_accuracy: 0.8512
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3513 - accuracy: 0.8722 - val_loss: 0.3587 - val_accuracy: 0.8725
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3241 - accuracy: 0.8813 - val_loss: 0.3402 - val_accuracy: 0.8788
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3073 - accuracy: 0.8864 - val_loss: 0.3304 - val_accuracy: 0.8800
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2915 - accuracy: 0.8917 - val_loss: 0.3229 - val_accuracy: 0.8817
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2781 - accuracy: 0.8963 - val_loss: 0.3232 - val_accuracy:

we're going to re-instantiate the model using that number of epochs

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/49
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5245 - accuracy: 0.8175 - val_loss: 0.4218 - val_accuracy: 0.8497
Epoch 2/49
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3906 - accuracy: 0.8602 - val_loss: 0.3828 - val_accuracy: 0.8617
Epoch 3/49
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3513 - accuracy: 0.8722 - val_loss: 0.3548 - val_accuracy: 0.8710
Epoch 4/49
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3243 - accuracy: 0.8821 - val_loss: 0.3373 - val_accuracy: 0.8817
Epoch 5/49
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3047 - accuracy: 0.8879 - val_loss: 0.3401 - val_accuracy: 0.8793
Epoch 6/49
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2938 - accuracy: 0.8922 - val_loss: 0.3515 - val_accuracy: 0.8728
Epoch 7/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2787 - accuracy: 0.8985 - val_loss: 0.3309 - val_accuracy:

now we can evaluate this hypermodel

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 5ms/step - loss: 0.5123 - accuracy: 0.8838
[test loss, test accuracy]: [0.51230388879776, 0.8838000297546387]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.

[link](https://www.tensorflow.org/tutorials/keras/keras_tuner#train_the_model)